# Perceptron no  lineal, de Widrow



## Big Picture

%%html
<img src= 'https://storage.googleapis.com/open-courses/imagenes-5c33/DeepLearningTimeline.jpg' />

Camino a Seguir


*   Perceptron de Rosenblatt
*   Perceptron de Widrow
*   Multiperceptron, backpropagation


## Objetivo desde el punto de vista Matemático

####Objetivo Primario

%%html
<img src= 'https://storage.googleapis.com/open-courses/imagenes-5c33/fuzzy.png' />

<p> El primitivo perceptron de Rosenblatt del año 1958 solo funcionaba cuando los positivos y negativos eran lineamente separables. Esta limitacion provenia de que el error era o el valor 0 o el valor 1 </p>

<p> El evolucionado perceptron de Widrow,  año 1960, acepta conjuntos de p8untos que no son perfectamente separables </p>

<p> ära la clasificacion binaria Widrow lo logra  permitiendo funciones de error que estén en el intervalo [0,1] </p>

####Objetivo Secundario

El sequndo objetivo en el caso que sean linealmente separables, **evitar** que  sucedan este tipo de soluciones donde la recta se "pega" a uno de los puntos ya que esa condición alcanzaba para que Rosenblatt se detuviera

%%html
<img src= 'https://storage.googleapis.com/open-courses/imagenes-5c33/recta_overfitera.jpg' />


## Trabajo de Bernard Widrow

Bernard Widrow (posee 95 años a marzo-2025), en video de 2015, explicando su algoritmo

[The LMS algorithm and ADALINE. Part I - The LMS algorithm](https://youtu.be/hc2Zj55j1zU)


parte II, haciendo una demo

[The LMS algorithm and ADALINE. Part II - ADALINE and memistor ADALINE](https://youtu.be/skfNlwEbqck)


## Concepto matemático de Perceptron

%%html
<img src='https://storage.googleapis.com/open-courses/imagenes-5c33/perceptron_elemental.jpg' />


%%html
<img src='https://storage.googleapis.com/open-courses/imagenes-5c33/activation_function.png' />

Funcion de activacion escalón  {0,1}


## Funciones de activacion

El primitivo perceptrón de Ronsenblatt solo permitía la función de activacion de escalon

El perceptron de Windrow permite funciones de activacion arbitrarias, en la media que sean derivables

Utilizaremos para clasificacion alguna de estas dos funciones de activacion


*  **logsig**   función logistica    R -> (0, 1 )
*  **tansig**  funcion logistica   R -> ( -1, 1 )




Utilizaremos para "Adaline"  esta funcion de activacion

*   **purelin**   recta identidad  R -> R




### **purelin** funcion Lineal Adaline

%%html
<img src='https://storage.googleapis.com/open-courses/imagenes-5c33/purelin.jpg' />

###  **logsig** funcion Logistica  R -> (0, 1 )

%%html
<img src='https://storage.googleapis.com/open-courses/imagenes-5c33/LogisticFunction.png' />

Derivación de la derivada de la función logistica

%%html
<img src='https://storage.googleapis.com/open-courses/imagenes-5c33/LogisticDerivative.jpg' />
<p>Derivada de la funcion logística, expresada en funcion de si misma</p>

%%html
<img src='https://storage.googleapis.com/open-courses/imagenes-5c33/logsig.jpg' />

### **tansig** tangente Hiperbolica   R -> (-1, 1 )

%%html
<img src='https://storage.googleapis.com/open-courses/imagenes-5c33/tanh-sigmoid.png' />'

%%html
<img src='https://storage.googleapis.com/open-courses/imagenes-5c33/tansig.jpg' />'

## Optimización  del Mean Squared Error

## Descenso del Gradiente

%%html
<img src='https://storage.googleapis.com/open-courses/imagenes-5c33/grad_descent_01.jpg' />

Se quiere optimizar

%%html
<img src='https://storage.googleapis.com/open-courses/imagenes-5c33/grad_descent_02.jpg' />

%%html
<img src='https://storage.googleapis.com/open-courses/imagenes-5c33/grad_descent_03.jpg' />

%%html
<img src='https://storage.googleapis.com/open-courses/imagenes-5c33/grad_descent_04.jpg' />

## Pseudocódigo del Algoritmo de Widrow

%%html
<img src='https://storage.googleapis.com/open-courses/imagenes-5c33/WidrowAlgoritmo.jpg' />

### Código en Python del Algoritmo de Widrow

Código en Python que grafica como va evolucionando la recta que separa a los positivos de los negativos

In [ ]:
# conexion al Google Drive
from google.colab import drive
drive.mount('/content/.drive')
!mkdir -p "/content/.drive/My Drive/DMA"
!mkdir -p "/content/buckets"
!ln -s "/content/.drive/My Drive/DMA" /content/buckets/b1

In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt
%matplotlib inline
from IPython import display
import time
from sklearn.preprocessing import StandardScaler

In [ ]:
# Ejemplo, usted deberá generar sus propios puntos en el plano
#  los que mandatoriamente deben ser separables por una recta
#  los puntos traen incorporada la dimension artificial x0
#  <x0, x1, x2>    donde    x0 = 1

registros = [ [1, 0.7, 1.3], [1, 2.0, 1.1], [1, 1.0, 1.9],
            [1, 3.0, 1.0], [1, 1.5, 2.1] ]
clases = [0,0,0,1,1]


In [ ]:
# Paso las listas a numpy
X = np.array(registros)
Y = np.array(clases)

In [ ]:
# Estandarizo X
X = StandardScaler().fit_transform(X)

# vuelvo a 1.0  x0
X[:,0] = 1.0

In [ ]:
# Tamano datos
X_row = X.shape[0]
X_col = X.shape[1]

In [ ]:
# definicion de las funciones de activacion
#  y sus derivadas

def func_eval(fname, x):
    match fname:
        case "purelin":
            y = x
        case "logsig":
            y = 1.0 / ( 1.0 + math.exp(-x) )
        case "tansig":
            y = 2.0 / ( 1.0 + math.exp(-2.0*x) ) - 1.0
    return y

def deriv_eval(fname, y):  #atencion que y es la entrada y=f( x )
    match fname:
        case "purelin":
            d = 1.0
        case "logsig":
            d = y*(1.0-y)
        case "tansig":
            d = 1.0 - y*y
    return d


In [ ]:
# definicion de la clase de graficos

class perceptron_plot:
    def __init__(self, X, Y, delay) -> None:
        self.X = X
        self.Y = Y
        self.delay = delay
        x1_min = np.min(X[:,1])
        x2_min = np.min(X[:,2])
        x1_max = np.max(X[:,1])
        x2_max = np.max(X[:,2])
        self.x1_min = x1_min - 0.75*(x1_max - x1_min)
        self.x1_max = x1_max + 0.75*(x1_max - x1_min)
        self.x2_min = x2_min - 0.75*(x2_max - x2_min)
        self.x2_max = x2_max + 0.75*(x2_max - x2_min)
        self.fig = plt.figure(figsize = (10,8))
        self.ax = self.fig.subplots()
        self.ax.set_xlim(self.x1_min, self.x1_max, auto=False)
        self.ax.set_ylim(self.x2_min, self.x2_max, auto=False)

    def graficar(self, W, epoch, fila, error) -> None:
        display.clear_output(wait =True)
        plt.cla()
        #self.ax = self.fig.subplots()

        self.ax.set_xlim(self.x1_min, self.x1_max)
        self.ax.set_ylim(self.x2_min, self.x2_max)
        plt.title( 'epoch ' + str(epoch) + '  reg ' + str(fila)  + ' error ' + str(error))
        # ploteo puntos positivos
        self.ax.plot(self.X[self.Y==1,1], self.X[self.Y==1,2], 'o', color="green", markersize=10)
        # ploteo puntos negativos
        self.ax.plot(self.X[self.Y==0,1], self.X[self.Y==0,2], 'o', color="blue", markersize=10)

        # Sobreploteo el punto que no coincidio
        if(fila>=0):
            self.ax.plot(self.X[fila,1], self.X[fila,2], 'o',
                         color= ('green' if self.Y[fila]==1 else 'blue'),
                         markersize= 12, markeredgecolor= 'red')

        #dibujo le recta
        vx2_min = -(W[1]*self.x1_min + W[0])/W[2]
        vx2_max = -(W[1]*self.x1_max + W[0])/W[2]

        self.ax.plot([self.x1_min, self.x1_max],
                     [vx2_min, vx2_max],
                     linewidth = 2,
                     color = 'red',
                     alpha = 0.5)

        display.display(plt.gcf())
        #plt.cla()
        time.sleep(self.delay)


    def graficarVarias(self, W, epoch, fila, error) -> None:
        display.clear_output(wait =True)
        plt.cla()
        #self.ax = self.fig.subplots()

        self.ax.set_xlim(self.x1_min, self.x1_max)
        self.ax.set_ylim(self.x2_min, self.x2_max)
        plt.title( 'epoch ' + str(epoch) + '  reg ' + str(fila) + ' error ' + str(error))
        # ploteo puntos positivos
        self.ax.plot(self.X[self.Y==1,1], self.X[self.Y==1,2], 'o', color="green", markersize=10)
        # ploteo puntos negativos
        self.ax.plot(self.X[self.Y==-1,1], self.X[self.Y==-1,2], 'o', color="blue", markersize=10)
        self.ax.plot(self.X[self.Y==0,1], self.X[self.Y==0,2], 'o', color="blue", markersize=10)

        # Sobreploteo el punto que no coincidio
        if(fila>=0):
            self.ax.plot(self.X[fila,1], self.X[fila,2], 'o',
                         color= ('green' if self.Y[fila]==1 else 'blue'),
                         markersize= 12, markeredgecolor= 'red')

        # dibujo las rectas
        for i in range(len(x0)):
            #vx2_min = -(W[0,i]*self.x1_min + x0[i])/W[1,i]
            #vx2_max = -(W[0,i]*self.x1_max + x0[i])/W[1,i]
            vx2_min = -(W[i,1]*self.x1_min + W[i,0])/W[i,2]
            vx2_max = -(W[i,1]*self.x1_max + W[i,0])/W[i,2]

            self.ax.plot([self.x1_min, self.x1_max],
                         [vx2_min, vx2_max],
                         linewidth = 2,
                         color = 'red',
                         alpha = 0.5)

        display.display(plt.gcf())
        #plt.cla()
        time.sleep(self.delay)


In [ ]:
# Incializo la recta azarosamente

# seteo de la semilla aleatoria
np.random.seed(102191) # mi querida random seed para que las corridas sean reproducibles

In [ ]:
# inicializo < x0, W >
W = np.array( np.random.uniform(-0.5, 0.5, size=X_col))

In [ ]:
# Limite de lo que estoy dispuesto a trabajar
epoch_limit = 500    # para terminar si no converge

In [ ]:
# cuando la mejora del error sea inferior a este valor, me detendré
error_delta_umbral = 0.0002

In [ ]:
# funcion activacion
funcion_activacion = 'logsig'  # uso la logistica

In [ ]:
# controla la velocidad de convergencia
learning_rate = 0.2

In [ ]:
# controla cada cuanto se grafica un epoch
demora_impresion = 0.5  # segundos

In [ ]:
# inicializaciones del bucle
epoch = 0

In [ ]:
# inicilizo el error
error_epoch = float('inf')
error_ant =  0.0

In [ ]:
# el bucle principal del algoritmo de Widrow

grafico = perceptron_plot(X=X, Y=Y, delay=demora_impresion)

# bucle principal del algoritmo de Widrow
# continuo mientras en la iteración anterior modifique algo  y NO llegué al límite de epochs
while ((math.fabs(error_epoch - error_ant) > error_delta_umbral) and (epoch < epoch_limit)):
    epoch += 1
    error_suma = 0.0
    error_ant = error_epoch

    # recorro siempre TODOS los registros de entrada
    for fila in range(X_row):
        # fila es el registro actual
        x = X[fila,]
        clase = Y[fila]
        # calculo el estimulo suma, producto interno
        estimulo = W @ x

        # funcion de activacion
        y = func_eval( funcion_activacion, estimulo)

        error = clase - y

        error_suma += error*error # acumulo el cuadrado de los errores

        gradiente = - 2.0 * error * deriv_eval(funcion_activacion,y)
        W = W -  ( learning_rate * gradiente * x )

        grafico.graficar(W, epoch, fila, error_epoch) # actualizo grafico


    error_epoch = error_suma / X_row # calculo el error promedio

In [ ]:
grafico = perceptron_plot(X, Y, 0)
grafico.graficar(W, epoch, -1, error_epoch)

In [ ]:
# el error
print("error_epoch= ", error_epoch)
print("error_ant = ", error_ant)
print("delta = ", math.fabs(error_epoch - error_ant))

In [ ]:
# la cantidad de epochs necesarias hasta encontrar una solucion
print("Para converger hicieron falta epochs=",epoch)

In [ ]:
# el vector solución W, que incluye a  X0
print( "La solucion es W = ", W)

In [ ]:
# la norma (longitud) del vector W
print( ' ║W║  = ' , np.linalg.norm(W) )

In [ ]:
# calculo la salida de la red
#  comprouebo que NO son valores 0 o 1
#  lo que implica que deberé decidir mediante un umbral

print( "fila\tclase\testimulo\ty")
for fila in range(X_row):
    # fila es el registro actual
    x = X[fila,]
    clase = Y[fila]
    # calculo el estimulo suma, producto interno
    estimulo = W @ x

    # funcion de activacion
    y = func_eval( funcion_activacion, estimulo)
    print( fila, "\t", clase, "\t", estimulo, "\t", y)